In [76]:
import hashlib
import time

In [94]:

class Block:
    '''index: The index or position of the block in the blockchain.
        proof_number: A value used in the proof-of-work algorithm to mine new blocks.
        previous_hash: The hash value of the previous block in the blockchain.
        data: The data or information stored in the block.
        timestamp: The timestamp indicating when the block was created.'''
    def __init__(self, index, proof_number, previous_hash, data, timestamp=None):
        self.index = index
        self.proof_number = proof_number
        self.previous_hash = previous_hash
        self.data = data
        self.timestamp = timestamp or time.time() #if time is provided, otherwise use time.time()

    @property
    def calculate_hash(self):
        '''calculates the hash value of the block by concatenating the block attributes 
        (index, proof_number, previous_hash, data, and timestamp) as a string and then applying 
        the SHA-256 hashing algorithm'''
        block_of_string = "{}{}{}{}{}".format(self.index, self.proof_number,
                                              self.previous_hash, self.data,
                                              self.timestamp)

        return hashlib.sha256(block_of_string.encode()).hexdigest() #hash value is returned as a hexadecimal string.

    def __repr__(self):
        '''Provides a string representation of the block object'''
        return "{} - {} - {} - {} - {}".format(self.index, self.proof_number,
                                               self.previous_hash, self.data,
                                               self.timestamp) #formatted string containing the values of the block attributes

class BlockChain:
    
    def __init__(self): # initialize the BlockChain object
        self.chain = []
        self.current_data = []
        self.nodes = set()
        self.construct_genesis()

    def construct_genesis(self):
        ''' first block in the blockchain'''
        self.construct_block(proof_number=0, previous_hash=0)

    def construct_block(self, proof_number, previous_hash):
        '''Creates a new Block object and adds it to the chain.
        
        Parameters:
            proof_number
            previous_hash
        '''
        block = Block(
            index=len(self.chain),
            proof_number=proof_number,
            previous_hash=previous_hash,
            data=self.current_data)
        self.current_data = []

        self.chain.append(block)
        return block

    @staticmethod
    def check_validity(block, previous_block):
        '''Checks the validity of a given block
        Parameters:
            block
            previous_block
            
         Checks to ensure the integrity and validity of the blockchain, 
         such as verifying the block's index, previous hash, proof of work, and timestamp
            '''
        if previous_block.index + 1 != block.index:
            return False

        elif previous_block.calculate_hash != block.previous_hash:
            return False

        elif not BlockChain.verifying_proof(block.proof_number,
                                            previous_block.proof_number):
            return False

        elif block.timestamp <= previous_block.timestamp:
            return False

        return True

    def new_data(self, sender, recipient, quantity):
        '''adds new transaction data to the current_data list
        
        Parameters:
            sender
            recipient
            quantity
        '''
        self.current_data.append({
            'sender': sender,
            'recipient': recipient,
            'quantity': quantity
        }) #Append to current data
        return True

    @staticmethod
    def proof_of_work(last_proof):
        '''implements a proof-of-work algorithm to find a valid proof number for mining a new block:
        
        The consensus identifies whether a number f' such that hash(ff') contain 4 leading zeroes
         f is the previous f' and
         f' is the new proof. 
         
         
         Parameters:
             last_proof
        '''
        proof_number = 0
        #Iterates through different proof numbers until a valid proof is found
        while BlockChain.verifying_proof(proof_number, last_proof) is False:
            proof_number += 1

        return proof_number

    @staticmethod
    def verifying_proof(last_proof, proof):
        #verifies the proof by checking if the hash of the concatenated
        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"

    @property
    def latest_block(self):
        '''Returns the latest block in the blockchain by accessing the last element of the chain list.'''
        return self.chain[-1]

    def block_mining(self, details_miner):
        '''Method that adds a new transaction data to the current_data list indicating that the miner has created a new block'''
        self.new_data(
            sender="0",  #it implies that this node has created a new block
            receiver=details_miner,
            quantity=
            1,  #create a new block/ identifying the proof number is awarded with 1
        )

        last_block = self.latest_block

        last_proof_number = last_block.proof_number #use last proof
        proof_number = self.proof_of_work(last_proof_number)

        last_hash = last_block.calculate_hash
        block = self.construct_block(proof_number, last_hash)

        return vars(block)

    @staticmethod
    def obtain_block_object(block_data):
        #obtains block object from the block data
        return Block(
            block_data['index'],
            block_data['proof_number'],
            block_data['previous_hash'],
            block_data['data'],
            timestamp=block_data['timestamp'])


In [95]:
blockchain = BlockChain()

print("***Starting Mining BXMCoin***")
print(blockchain.chain)

last_block = blockchain.latest_block
last_proof_number = last_block.proof_number
proof_number = blockchain.proof_of_work(last_proof_number)

blockchain.new_data(
    sender="0",  #Indicates the node created a new block
    recipient="John Doe",  #Send John Doe Coins
    quantity=
    1,  #creating a new block (or identifying the proof number) is awarded with 1
)

last_hash = last_block.calculate_hash
block = blockchain.construct_block(proof_number, last_hash)

print("***BXMCoin mined successfully***")
print(blockchain.chain)

***Starting Mining BXMCoin***
[0 - 0 - 0 - [] - 1687201234.87753]
***BXMCoin mined successfully***
[0 - 0 - 0 - [] - 1687201234.87753, 1 - 88914 - 2b04ae74d56be18e958396a141ba1a7277adbf65d4bf2a26cb96d99f9a2230bc - [{'sender': '0', 'recipient': 'John Doe', 'quantity': 1}] - 1687201235.3415403]


In [59]:
str(block).split('-')

['10 ',
 ' 101132 ',
 ' 12715e4be3aaab6a51a18b8b48cbc20d6892e2ff0f958ceaf08d5736b1979847 ',
 " [{'sender': 'Julie Lopez', 'recipient': 'Rebecca Evans', 'quantity': 1}] ",
 ' 1687191557.0949965']

In [61]:
str(block).split('-')[4].strip()

'1687191557.0949965'

##### Simulate 1000 transactions

In [68]:
%%time
import pandas as pd
import time
from faker import Faker
import random

fake = Faker()

# Create an empty list to store transaction details
transaction_data = []

blockchain = BlockChain()

print("***Starting Mining BXMCoin***")
print(blockchain.chain)

for i in range(1000):
    start_time = time.time()  # Start measuring transaction time
    
    sender_name = fake.name()
    recipient_name = fake.name()

    quantity = random.randint(1, 100)  # Generate a random quantity between 1 and 20

    blockchain.new_data(
        sender=sender_name,
        recipient=recipient_name,
        quantity=quantity
    )

    last_block = blockchain.latest_block
    last_proof_number = last_block.proof_number
    proof_number = blockchain.proof_of_work(last_proof_number)

    last_hash = last_block.calculate_hash
    block = blockchain.construct_block(proof_number, last_hash)

    end_time = time.time()  # Stop measuring transaction time
    time_taken = end_time - start_time  # Calculate the time taken for the transaction
    
    # Append transaction details to the list
    transaction_data.append({
        'Index': str(block).split('-')[0].strip(),
        'Sender': sender_name,
        'Recipient': recipient_name,
        'Proof Number': str(block).split('-')[1].strip(),
        'Quantity': quantity,
        'Token/Hash': str(block).split('-')[2].strip(),
        'Timestamp': str(block).split('-')[4].strip(),
        'Time Taken': time_taken,
        'Data': str(block).split('-')[3].strip()
    })

#print("***BXMCoin mined successfully***")
#print(blockchain.chain)

# Create DataFrame from the transaction data
transaction_df = pd.DataFrame(transaction_data)

# Print the transaction details DataFrame
transaction_df.head()

***Starting Mining BXMCoin***
[0 - 0 - 0 - [] - 1687192097.259176]
CPU times: total: 5min 9s
Wall time: 6min 7s


Index           Sender        Recipient Proof Number  Quantity   
0     1  Victoria Glover    Nathan Steele        88914        57  \
1     2      Joseph Wall       Wendy Long        49714        68   
2     3     Austin Rivas        Rita Neal        61095        94   
3     4      Cindy Booth  Adrian Gonzalez        52386        85   
4     5      Curtis Wade    Meghan Osborn        37083        24   

                                          Token/Hash           Timestamp   
0  3ee571684777beeaa40625f0d4f86c4f944f0e0dd9f798...  1687192097.7681782  \
1  7d75d064270274893c34f10241ec24db68a0cbe7938298...  1687192098.0441656   
2  0a5a825d92a68d65ce2d41e1bf0743682b89a205c7481d...   1687192099.244235   
3  d1d5053d16101fd656994fea58a037a95fc84c8012d0c0...   1687192099.856235   
4  14e846ddaf3d7ba8e47f4c94eb22819bae1d0df3a42a95...  1687192100.1032352   

   Time Taken                                               Data  
0    0.508002  [{'sender': 'Victoria Glover', 'recipient': 'N...  
1    0.275987  [{'sender': 'Joseph Wall', 'recipient': 'Wendy...  
2    1.199071  [{'sender': 'Austin Rivas', 'recipient': 'Rita...  
3    0.612000  [{'sender': 'Cindy Booth', 'recipient': 'Adria...  
4    0.247000  [{'sender': 'Curtis Wade', 'recipient': 'Megha...

In [96]:
transaction_df.head().to_csv('top crypto.csv', index = False)

In [86]:
#Check for duplicate transactions
transaction_df[transaction_df[['Token/Hash']].duplicated()]

Empty DataFrame
Columns: [Index, Sender, Recipient, Proof Number, Quantity, Token/Hash, Timestamp, Time Taken, Data]
Index: []

In [89]:
# Create the line plot using Plotly
fig = go.Figure(data=go.Scatter(x=transaction_df.index, y=transaction_df['Timestamp'], mode='lines'))

# Set plot labels and title
fig.update_layout(
    xaxis_title='Transaction',
    yaxis_title='Timestamp',
    title='Timestamp of Transactions'
)

# Display the plot
fig.show()

In [73]:
#summary statistics
transaction_df[['Time Taken', 'Quantity']].describe()

Time Taken     Quantity
count  1000.000000  1000.000000
mean      0.366567    49.275000
std       0.381382    29.245866
min       0.001001     1.000000
25%       0.097739    24.000000
50%       0.246003    49.000000
75%       0.497992    75.000000
max       2.258074   100.000000

In [74]:
#correlation
transaction_df[['Time Taken', 'Quantity']].corr()

Time Taken  Quantity
Time Taken    1.000000 -0.014495
Quantity     -0.014495  1.000000

In [79]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Histogram(x=transaction_df['Time Taken'], nbinsx=100)])

# Set plot labels and title
fig.update_layout(
    xaxis_title='Time Taken',
    yaxis_title='Frequency',
    title='Distribution of Time Taken to mine'
)

# Display the plot
fig.show()

##### Simulate 3000 transactions

In [97]:
%%time
import pandas as pd
import time
from faker import Faker
import random

fake = Faker()

# Create an empty list to store transaction details
transaction_data = []

blockchain = BlockChain()

print("***Starting Mining BXMCoin***")
print(blockchain.chain)

for i in range(2000):
    start_time = time.time()  # Start measuring transaction time
    
    sender_name = fake.name()
    recipient_name = fake.name()

    quantity = random.randint(1, 100)  # Generate a random quantity between 1 and 20

    blockchain.new_data(
        sender=sender_name,
        recipient=recipient_name,
        quantity=quantity
    )

    last_block = blockchain.latest_block
    last_proof_number = last_block.proof_number
    proof_number = blockchain.proof_of_work(last_proof_number)

    last_hash = last_block.calculate_hash
    block = blockchain.construct_block(proof_number, last_hash)

    end_time = time.time()  # Stop measuring transaction time
    time_taken = end_time - start_time  # Calculate the time taken for the transaction
    
    # Append transaction details to the list
    transaction_data.append({
        'Index': str(block).split('-')[0].strip(),
        'Sender': sender_name,
        'Recipient': recipient_name,
        'Proof Number': str(block).split('-')[1].strip(),
        'Quantity': quantity,
        'Token/Hash': str(block).split('-')[2].strip(),
        'Timestamp': str(block).split('-')[4].strip(),
        'Time Taken': time_taken,
        'Data': str(block).split('-')[3].strip()
    })

#print("***BXMCoin mined successfully***")
#print(blockchain.chain)

# Create DataFrame from the transaction data
transaction_df = pd.DataFrame(transaction_data)

# Print the transaction details DataFrame
transaction_df.head()

***Starting Mining BXMCoin***
[0 - 0 - 0 - [] - 1687203063.0052156]
CPU times: total: 9min 8s
Wall time: 9min 42s


Index              Sender       Recipient Proof Number  Quantity   
0     1        David Lucero     Shannon Fox        88914        86  \
1     2   Charles Robertson   Joshua Weaver        49714        44   
2     3  Catherine Anderson    Shawn Fisher        61095        47   
3     4      Charles Bowman   Antonio Smith        52386        39   
4     5         Emily Smith  Kevin Faulkner        37083        30   

                                          Token/Hash           Timestamp   
0  816929a5bcf5a09b7227d6712b25b7cbdbd0d0947b6bda...  1687203063.4551396  \
1  d79dac95e89895d823af91ea652cb5d018f8e00a5d549c...   1687203063.714145   
2  343baa45e85bd784f34f0f1ea641100a5e87075d96b192...  1687203064.0641549   
3  29817ed4ec20471fe37b103af9ec7e1334ed6ce1c8c748...  1687203064.3271396   
4  3adfa8e2f745db9bfca0bfce362fa73e54fb2d12faf7cc...  1687203064.5121548   

   Time Taken                                               Data  
0    0.449924  [{'sender': 'David Lucero', 'recipient': 'Shan...  
1    0.259005  [{'sender': 'Charles Robertson', 'recipient': ...  
2    0.350010  [{'sender': 'Catherine Anderson', 'recipient':...  
3    0.262985  [{'sender': 'Charles Bowman', 'recipient': 'An...  
4    0.185015  [{'sender': 'Emily Smith', 'recipient': 'Kevin...

In [98]:
#Check for duplicate transactions
transaction_df[transaction_df[['Token/Hash']].duplicated()]

Empty DataFrame
Columns: [Index, Sender, Recipient, Proof Number, Quantity, Token/Hash, Timestamp, Time Taken, Data]
Index: []

In [99]:
# Create the line plot using Plotly
fig = go.Figure(data=go.Scatter(x=transaction_df.index, y=transaction_df['Timestamp'], mode='lines'))

# Set plot labels and title
fig.update_layout(
    xaxis_title='Transaction',
    yaxis_title='Timestamp',
    title='Timestamp of Transactions'
)

# Display the plot
fig.show()

In [100]:
#summary statistics
transaction_df[['Time Taken', 'Quantity']].describe()

Time Taken     Quantity
count  2000.000000  2000.000000
mean      0.290950    50.813000
std       0.307187    28.688509
min       0.000000     1.000000
25%       0.070007    26.000000
50%       0.190000    51.000000
75%       0.398253    75.000000
max       3.227992   100.000000

In [101]:
#correlation
transaction_df[['Time Taken', 'Quantity']].corr()

Time Taken  Quantity
Time Taken    1.000000 -0.017876
Quantity     -0.017876  1.000000

In [102]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Histogram(x=transaction_df['Time Taken'], nbinsx=100)])

# Set plot labels and title
fig.update_layout(
    xaxis_title='Time Taken',
    yaxis_title='Frequency',
    title='Distribution of Time Taken to mine'
)

# Display the plot
fig.show()